# BibTeX to Markdown Converter

This program converts a BibTeX data file into a formatted paper list in Markdown. BibTeX is widely used for referencing in academic papers, but Markdown is a simpler, more readable format for displaying lists of references on web pages or in documents. By converting BibTeX entries to Markdown, you can easily present bibliographic information in a clean way.

Example
Given the following BibTeX entry:
```bibtex
@article{karpathy2015visualizing,
  title={Visualizing and understanding recurrent networks},
  author={Karpathy, Andrej and Johnson, Justin and Fei-Fei, Li},
  journal={arXiv preprint arXiv:1506.02078},
  year={2015}
}
```

The program will produce the following Markdown formatted paper list:

**Visualizing and understanding recurrent networks**. Karpathy, Andrej, Justin Johnson, and Li Fei-Fei. *arXiv preprint arXiv:1506.02078* (2015).


Get an API key from the links below

https://www.scraperapi.com/blog/best-google-scholar-apis-proxies/

https://serpapi.com/google-scholar-organic-results

https://serpdog.io/google-scholar-api/

https://www.scaleserp.com/ 

In [3]:
import requests
import bibtexparser

'# References\n\n- **Authorship attribution** by Holmes, David I, 1994\n- **Combating Misinformation in the Age of LLMs: Opportunities and Challenges** by Canyu Chen and Kai Shu, 2023\n- **The state of authorship attribution studies: Some problems and solutions** by Rudman, Joseph, 1997\n- **Machine learning in automated text categorization** by Sebastiani, Fabrizio, 2002\n- **A survey of modern authorship attribution methods** by Stamatatos, Efstathios, 2009\n- **Adversarial stylometry: Circumventing authorship recognition to preserve privacy and anonymity** by Brennan, Michael and Afroz, Sadia and Greenstadt, Rachel, 2012\n- **Authorship verification: a review of recent advances** by Stamatatos, Efstathios, 2016\n- **A Survey on Authorship Analysis Tasks and Techniques** by Misini, Arta and Kadriu, Arbana and Canhasi, Ercan, No year\n- **Code authorship attribution: Methods and challenges** by Kalgutkar, Vaibhavi and Kaur, Ratinder and Gonzalez, Hugo and Stakhanova, Natalia and Matyu

In [27]:
# https://docs.serpdog.io/google-scholar-api
def get_paper_link(title, authors):
    api_key = 'API'  # Replace with your Serpdog API key
    params = {
        "api_key": api_key,
        "q": f"{title} {authors}"
    }
    
    response = requests.get("https://api.serpdog.io/scholar", params=params)

    
    if response.status_code == 200:
        results = response.json().get("scholar_results", [])
        if results:
            # Assuming the first result is the most relevant one
            resource_link = results[0].get("resources", [])
            if resource_link:
                return resource_link[0].get("link", "No link found")
            else:
                return results[0].get("title_link", "No link found")
        else:
            return "No results found"
    else:
        return f"Error: {response.status_code}"

# for paper in papers[:3]:
#     title = paper.get('title', 'No Title').strip('{}').replace('\n', ' ')
#     authors = format_authors(paper.get('author', 'No Author'))
#     venue = paper.get('booktitle', paper.get('journal', 'No venue'))
#     year = paper.get('year', 'No Year')
#     link = get_paper_link(title, authors)
#     print(f"{title} {authors}. link: {link}")

In [36]:
# Function to extract bibtex entries from a file
def extract_bibtex_entries(file_path):
    with open(file_path) as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)
    return bib_database.entries


# Function to format authors
def format_authors(authors):
    author_list = authors.split(' and ')
    formatted_authors = []
    for author in author_list:
        names = author.split(', ')
        if len(names) == 2:
            formatted_authors.append(f"{names[1]} {names[0]}")
        else:
            formatted_authors.append(names[0])
    if len(formatted_authors) > 1:
        return ', '.join(formatted_authors[:-1]) + ', and ' + formatted_authors[-1]
    else:
        return formatted_authors[0]


# Function to generate markdown for the papers
def generate_markdown(papers):
    markdown = "# Survey Paper List\n\n"
    for i, paper in enumerate(papers[290:]):
        title = paper.get('title', 'No Title').strip('{}').replace('\n', ' ')
        authors = format_authors(paper.get('author', 'No Author'))
        venue = paper.get('booktitle', paper.get('journal', 'No venue'))
        year = paper.get('year', 'No Year')
        # link = ""
        link = get_paper_link(title, authors)
        markdown += f"- **{title}.** {authors}. *{venue}* ({year}) [[link]]({link})\n"
        print(f"Index: {i}, title: {title}. link: {link}")
    return markdown


# Path to your .bib file
bib_file_path = 'tmp.bib'

# Extract bibtex entries
papers = extract_bibtex_entries(bib_file_path)

# Generate markdown
markdown_content = generate_markdown(papers)

with open('paper_list_rest.md', 'w') as f:
    f.write(markdown_content)

len(papers)

<Response [200]>
Index: 0, title: Fake news detection and fact verification using knowledge graphs and machine learning. link: https://www.researchgate.net/profile/Danish-Shakeel-2/publication/348931042_Fake_news_detection_and_fact_verification_using_knowledge_graphs_and_machine_learning/links/6017de6892851c2d4d0aac7f/Fake-news-detection-and-fact-verification-using-knowledge-graphs-and-machine-learning.pdf
<Response [200]>
Index: 1, title: The limitations of stylometry for detecting machine-generated fake news. link: https://direct.mit.edu/coli/article-abstract/46/2/499/93369
<Response [200]>
Index: 2, title: Llmdet: A third party large language models generated text detection tool. link: https://arxiv.org/abs/2305.15004
<Response [200]>
Index: 3, title: Spotting LLMs With Binoculars: Zero-Shot Detection of Machine-Generated Text. link: https://arxiv.org/abs/2401.12070
<Response [200]>
Index: 4, title: Fast-detectgpt: Efficient zero-shot detection of machine-generated text via conditio

341